In [ ]:
using InteractiveUtils, Pkg, Test

Base.banner()

Pkg.activate(@__DIR__)
Pkg.status()

using JET

# Basic Usage

- Interactive entry point: `[@]report_call`, `[@]report_opt`
  - `report_call` for detecting bugs
  - `report_opt` for detecting performance problems
- Works like `[@]code_typed`

In [ ]:
report_call(sum, (Vector{Int},))

In [ ]:
@report_call sum("julia")

# Notable features for package development

- Test.jl integration
- `report_package`
- The typo detection mode
- Report filters

# Test.jl integration

JET provides smooth integration with Test.jl:
- Allows users to use JET in their test suite
  - `[@]report_call`, `[@]report_opt` <-> `[@]test_call`, `[@]test_opt`
  - `report_package`, `report_file` <-> `test_package`, `test_file`
- Results are included in a test set summary
- Also supports Test.jl features like `broken` and `skip`

In [ ]:
function test_func(a)
    return a + 1
end

@test_call test_func("julia")

In [ ]:
@testset "JET code quality test" begin
    @test_call test_func("julia")
    @test_call test_func(1)
end

In [ ]:
@testset "JET code quality test" begin
    @test_call skip=true test_func("julia")
    @test_call test_func(1)
end

Packages using JET in their CI in this way: 
- JET: <https://github.com/aviatesk/JET.jl/blob/master/test/self_check.jl>
- CliMA ecosystem

# `report_package`

`report_package` is an easy entry point to analyze package code.

- Interactive entry points (`report_call`)
  - Bad: Require programmers to give concrete input types
  - Good: More accurate
- `report_package`: Use method signatures as analysis entry point
  - Good: Easy to use
  - Bad: Less accurate

Ideally package test suite should use the interactive entry points if possible.
`report_package` is still useful for a first inspection, etc.

Improved affinity with Pkg.jl system: 

No need to instantiate/activate a target package environment beforehand.
Just install it, and give its name to `report_package`.

In [ ]:
report_package("Demo1")

In [ ]:
# comparison with interactive entry points
using Demo1

# @report_call callsum2("julia")
@report_call callsum3("julia")

`target_defined_modules` configuration: 
- Only report problems that are detected within the module context of a target package.
- Allows you to focus on your code.

In [ ]:
# TODO-1 use `target_defined_modules=true` and filter out errors from `Core.Compiler`
# TODO-2 put the configuration into a configuration file

report_package("Demo2")

# Configurations

- Sometimes JET produces very noisy results. `mode=:typo` only detects typos, so can be useful as a first step to improve your code.
- `JET.configured_reports` can be used to implement your own report filter.

In [ ]:
report_package("JuliaHub"; toplevel_logger=nothing)

In [ ]:
report_package("JuliaHub"; mode=:typo, toplevel_logger=nothing)

In [ ]:
result = report_package("JuliaHub"; target_defined_modules=true, toplevel_logger=nothing)

In [ ]:
reports = JET.get_reports(result)

In [ ]:
unique(typeof.(reports))

In [ ]:
reports[end].vst[end].linfo.def.module

In [ ]:
reports[end].t

In [ ]:
struct MyFilter end
function JET.configured_reports(::MyFilter, reports::Vector{JET.InferenceErrorReport})
    filter(reports) do report
        # XXX this is necessary since a custom `report_config` overrides `target_defined_modules` setting.
        occursin("JuliaHub", string(last(report.vst).linfo.def.module)) || return false

        isa(report, JET.MethodErrorReport) || return false
        report.t isa DataType || return false
        any(==(Nothing), report.t.parameters) && return false
        return true
    end
end

result = report_package("JuliaHub"; 
                        report_config=MyFilter(), 
                        toplevel_logger=nothing)